In [48]:
import tensorflow as tfflow
from sklearn.manifold import TSNE
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
import pickle
import numpy as np
import pandas as pd



Reading Label Data

In [49]:
df_product_detailed = pd.read_csv('clean_data/cleaned_products_detailed.csv')
df_product_standard = pd.read_csv('clean_data/cleaned_products.csv')

df_product_standard = df_product_standard[["ctr_product_num", "short_desc"]]

In [50]:
embeddings = ['TFIDF_all.csv','meta_prod2vec_all.csv','minilm_100k.csv','onehot_all.csv','prod2vec_all.csv','roberta_100k.csv','t5_100k.csv']

# Generate visualization

In [51]:
for i, filename in enumerate(embeddings):
    test_embedding = pd.read_csv("embeddings/" + filename, index_col=0, header=0)
    test_embedding = test_embedding.sample(frac=1)[:1000]
    df_combined = test_embedding.join(df_product_standard.set_index("ctr_product_num"))
    df_combined.dropna(inplace=True)

    #convert target column to numerical
    #df_combined.merch_lob_nm = pd.Categorical(pd.factorize(df_combined.merch_lob_nm)[0])

    drop_for_X = ["short_desc"]

    X = df_combined.drop(columns=drop_for_X)
    Y = df_combined[["short_desc"]]

    # Save Labels separately on a line-by-line manner.
    with open('visualization/'+filename+'_label.tsv', "w") as f:
        for i in Y['short_desc']:
            f.write("{}\n".format(i))

    # Convert the product embeddings to a numpy array
    product_embeddings_array = np.array(X, dtype=np.float32)

    # Use t-SNE to reduce the dimensions of the product embeddings
    tsne = TSNE(perplexity=30,n_components=3)
    product_embeddings_3d = tsne.fit_transform(product_embeddings_array)
    #print(product_embeddings_array)
    #print(product_embeddings_3d)
    # Save the dataset to a file
    with open('visualization/'+filename+'_value.tsv', 'w') as f:
        for x, y, z in product_embeddings_3d:
            f.write(f"{x}\t{y}\t{z}\n")

# For store embedding

In [52]:
store_sales_embedding = pd.read_csv('embeddings/store_embeddings/store_sales_embedding.csv')

path_store = "clean_data/cleaned_store.csv" #Eric
# put it in data farme
df_store = pd.read_csv(path_store, index_col=0, header=0)
df_store_label = df_store[['store_num','store_nm']]
df_store_label

store_embedding = store_sales_embedding.drop_duplicates(subset="store_num")
store_embedding = store_embedding.drop(columns=["yr_num","wk_num", "sales_qty"], axis=1)
store_embedding = store_embedding.sort_values(by="store_num")
store_embedding.reset_index(inplace=True, drop=True )

In [53]:
merged_data = pd.merge(store_embedding, df_store_label, on='store_num', how='left')

In [54]:
#shuffle the data
df = merged_data.sample(frac=1)
drop_for_X = ['store_nm']

X = df.drop(columns=drop_for_X)
Y = df[["store_nm"]]

In [55]:
# Save Labels separately on a line-by-line manner.
with open('visualization/'+'store'+'_label.tsv', "w") as f:
    for i in Y['store_nm']:
        f.write("{}\n".format(i))

# Convert the product embeddings to a numpy array
product_embeddings_array = np.array(X, dtype=np.float32)

# Use t-SNE to reduce the dimensions of the product embeddings
tsne = TSNE(perplexity=30,n_components=3)
product_embeddings_3d = tsne.fit_transform(product_embeddings_array)
#print(product_embeddings_array)
#print(product_embeddings_3d)
# Save the dataset to a file
with open('visualization/'+'store'+'_value.tsv', 'w') as f:
    for x, y, z in product_embeddings_3d:
        f.write(f"{x}\t{y}\t{z}\n")